In [48]:
# importing required classes
from pypdf import PdfReader, PdfWriter

In [49]:
import xlsxwriter, pandas as pd

In [50]:
composicoes = {
    "servico_title": [
        "servico",
        "descricao",
        "producao",
        "unidade",
        "custo unitario execucao",
        "custo direto",
        "bdi",
        "preco unitario total",
        "pagina",
    ],
    "equipamentos_title": [
        "servico",
        "equipamento",
        "codigo",
        "ut. pr.",
        "ut. impr.",
        "vl. hr. prod.",
        "vl. hr. impr.",
        "consumo",
        "custo horario",
    ],
    "mao_de_obra_title": [
        "servico",
        "mao de obra",
        "codigo",
        "eq. salarial",
        "sal. hora",
        "% encargos",
        "consumo",
        "custo horario",
    ],
    "itens_de_incidencia_title": [
        "servico",
        "itens de incidencia",
        "codigo",
        "%",
        "M.O.",
        # "equip.",
        # "mat.",
        "custo",
    ],
    "materiais_title": [
        "servico",
        "materiais",
        "codigo",
        "unidade",
        "custo unitario",
        "consumo",
        "custo",
    ],
    "servicos_title": [
        "servico",
        "servicos",
        "codigo",
        "unidade",
        "custo unitario",
        "consumo",
        "custo",
    ],
    "itens_de_transporte_title": [
        "servico",
        "itens de transporte",
        "codigo",
        "unidade",
        "formula",
        "x1",
        "x2",
        "x3",
        "custo unitario",
        "consumo",
        "custo",
    ],
}

In [51]:
def add_sheet(wb, sheet_name, sheet_title):
    ws = wb.add_worksheet(sheet_name)
    for n, i in enumerate(sheet_title):
        ws.write(0, n, i)

In [52]:
def add_data_sheet(wb, sheet_name, sheet_vars):
    row = 1
    ws = wb.get_worksheet_by_name(sheet_name)
    for i in sheet_vars:
        for n, v in enumerate(i):
            ws.write(row, n, v)
        row += 1

In [180]:
composicoes_files = [
    # "source/composicoes.pdf",
    "source/composicoes---t109.pdf",
]

global_composicoes = {}
for composicoes_file in composicoes_files:  # creating a pdf reader object
    global_servico = []
    global_equipamentos = []
    global_mao_de_obra = []
    global_itens_de_incidencia = []
    global_producao_equipe = []
    global_materiais = []
    global_servicos = []
    global_itens_de_transporte = []
    global_pages = {}

    target = composicoes_file.replace("source", "target").replace(".pdf", ".xlsx")
    # print(target)
    wb = xlsxwriter.Workbook(target)
    for k, v in composicoes.items():
        sheet = k.replace("_title", "").replace("_", " ")
        # print(sheet, v)
        add_sheet(wb, sheet, v)

    reader = PdfReader(composicoes_file)
    # print((reader.pages))

    for n, page in enumerate(reader.pages[:60]):
        # extracting text from page
        texts = page.extract_text().splitlines()

        is_a = False
        text_equip = ""
        is_b = False
        text_mdo = ""
        have_encargos = False
        is_c = False
        is_not_c = False
        is_d = False
        is_e = False
        is_f = False
        is_g = False
        is_h = False
        is_not_h = False
        equipamento_s = []
        mao_de_obra_s = []
        itens_de_incidencia_s = []
        producao_equipe_s = []
        materiais_s = []
        servicos_s = []
        itens_de_transporte_s = []
        for text in texts:
            text_left = text.lstrip().rstrip()
            print(text_left)
            if text_left.startswith("Serviço"):
                servico = text_left[
                    text_left.find("Serviço")
                    + len("Serviço")
                    + 2 : text_left.find("Serviço")
                    + len("Serviço")
                    + 7
                ].strip()
                servico_nome = text_left[
                    text_left.find("Serviço")
                    + len("Serviço")
                    + 9 : text_left.find("Unidade")
                    - 1
                ].strip()
                servico_page = {servico: n}
                global_pages.update(servico_page)
                _composicoes = {composicoes_file: global_pages}
                global_composicoes.update(_composicoes)
            if "Unidade: " in text_left:
                unidade = text_left[
                    text_left.find("Unidade") + len("Unidade") + 2 :
                ].strip()
                # print(servico, unidade)
            if text_left.startswith("(A)"):
                is_a = True
            if text_left.startswith("(B)"):
                is_b = True
            if "Encargos(%)" in text_left:
                have_encargos = True
            if text_left.startswith("(C)"):
                is_c = True
            if text_left.startswith("Custo Horário"):
                is_not_c = True
            if text_left.startswith("(D)"):
                is_d = True
            if text_left.startswith("(E)"):
                is_e = True
            if text_left.startswith("(F)"):
                is_f = True
            if text_left.startswith("(G)"):
                is_g = True
            if text_left.startswith("(H)"):
                is_h = True
            if text_left.startswith("Custo Direto"):
                is_not_h = True
            if not (text_left.startswith("auxiliar")):
                if is_a == True and is_b == False:
                    if not text_left.startswith("(A)"):
                        _equipamento = text.split()
                        if (
                            "," not in _equipamento[-1:][0]
                            and "." not in _equipamento[-1:][0]
                        ):
                            text_equip = text_left
                        else:
                            equipamento = [
                                " ".join(
                                    filter(
                                        None,
                                        [text_equip]
                                        + _equipamento[
                                            : len(_equipamento)
                                            - (7 if text_equip == "" else 6)
                                        ],
                                    )
                                )
                            ] + _equipamento[-7:]
                            if text_equip != "":
                                equipamento[0] = equipamento[0][:-5]
                                equipamento[1] = equipamento[1][-5:]
                            equipamento_s.append(equipamento)
                            text_equip = ""
                if is_b == True and is_c == False:
                    if not text_left.startswith("(B)"):
                        _mao_de_obra = text.split()
                        print(_mao_de_obra)
                        if (
                            "," not in _mao_de_obra[-1:][0]
                            and "." not in _equipamento[-1:][0]
                        ):
                            text_mdo = text_left
                        else:
                            mao_de_obra = [
                                " ".join(
                                    filter(
                                        None,
                                        [text_mdo]
                                        + _mao_de_obra[
                                            : len(_mao_de_obra)
                                            - (5 if text_mdo == "" else 4)
                                            - (1 if have_encargos == True else 0)
                                        ],
                                    )
                                )
                            ] + _mao_de_obra[-5 - (1 if have_encargos == True else 0) :]
                            print("----> mdo:", mao_de_obra)
                            if have_encargos == False:
                                mao_de_obra.insert(4, "-")
                            if text_mdo != "":
                                mao_de_obra[0] = mao_de_obra[0][:-5]
                                mao_de_obra[1] = mao_de_obra[1][-5:]
                            mao_de_obra_s.append(mao_de_obra)
                            text_mdo = ""
                if is_c == True and is_not_c == False:
                    if not text_left.startswith("(C)") and not text_left.startswith(
                        "Custo Horário"
                    ):
                        _itens_de_incidencia = text.split()
                        itens_de_incidencia = [
                            " ".join(
                                _itens_de_incidencia[: len(_itens_de_incidencia) - 4]
                            )
                        ] + _itens_de_incidencia[-4:]
                        itens_de_incidencia_s.append(itens_de_incidencia)
                if is_f == True and is_g == False:
                    if not text_left.startswith("(F)"):
                        _materiais = text.split()
                        materiais = [
                            " ".join(_materiais[: len(_materiais) - 5])
                        ] + _materiais[-5:]
                        materiais_s.append(materiais)
                if is_g == True and is_h == False:
                    if not text_left.startswith("(G)"):
                        _servicos = text.split()
                        servicos = [
                            " ".join(_servicos[: len(_servicos) - 5])
                        ] + _servicos[-5:]
                        servicos_s.append(servicos)
                if is_h == True and is_not_h == False:
                    if not text_left.startswith("(H)") and not text_left.startswith(
                        "Custo Direto"
                    ):
                        _itens_de_transporte = text.split()
                        itens_de_transporte = [
                            " ".join(
                                _itens_de_transporte[: len(_itens_de_transporte) - 9]
                            )
                        ] + _itens_de_transporte[-9:]
                        itens_de_transporte_s.append(itens_de_transporte)
                if text_left.startswith("(D) Produção da Equipe"):
                    producao = text_left[
                        text_left.find("(D) Produção da Equipe")
                        + len("(D) Produção da Equipe")
                        + 1 :
                    ].strip()
                if text_left.startswith("(E)"):
                    custo_unitario_execucao = (
                        text_left[text_left.find("(E)") + len("(E)") + 1 :]
                        .strip()
                        .split()[-1]
                    )
                if text_left.startswith("Custo Direto"):
                    custo_direto = (
                        text_left[
                            text_left.find("Custo Direto") + len("Custo Direto") + 1 :
                        ]
                        .strip()
                        .split()[-1]
                    )
                if text_left.startswith("BDI"):
                    bdi = (
                        text_left[text_left.find("BDI") + len("BDI") + 1 :]
                        .strip()
                        .split()[-1]
                    )
                if text_left.startswith("Preço Unitário"):
                    preco_unitario_total = (
                        text_left[
                            text_left.find("Preço Unitário")
                            + len("Preço Unitário")
                            + 1 :
                        ]
                        .strip()
                        .split()[-1]
                    )
                    # unidade = text_left[text_left.find("Unidade") + len("Unidade") + 2 :]
                    servicos = [
                        servico,
                        servico_nome,
                        producao,
                        unidade,
                        custo_unitario_execucao,
                        custo_direto,
                        bdi,
                        preco_unitario_total,
                        n + 1,
                    ]

        global_servico.append(servicos)
        for equipamento in equipamento_s:
            equipamento = [servico] + equipamento
            global_equipamentos.append(equipamento)
        for i in mao_de_obra_s:
            i = [servico] + i
            global_mao_de_obra.append(i)
        for i in itens_de_incidencia_s:
            i = [servico] + i
            global_itens_de_incidencia.append(i)
        for i in producao_equipe_s:
            i = [servico] + i
            global_producao_equipe.append(i)
        for i in materiais_s:
            i = [servico] + i
            global_materiais.append(i)
        for i in servicos_s:
            i = [servico] + i
            global_servicos.append(i)
        for i in itens_de_transporte_s:
            i = [servico] + i
            global_itens_de_transporte.append(i)
    for k, v in composicoes.items():
        add_data_sheet(
            wb,
            k.replace("_title", "").replace("_", " "),
            globals()[f"global_{k.replace('_title', '')}"],
        )
    wb.close()

AGETOP - AGÊNCIA GOIANA DE TRANSPORTES E OBRAS - AGETOP 24/01/2014 - 16:30
Relatório de Composição do Serviço
Página: 1 de 650
Tabela de Preços: TABELA DE TERRAPLENAGEM, PAVIMENTAÇÃO E OBRAS DE ARTE ESPECIAIS - DESONERADA - JUNHO
DE 2013- (T109) Data base: 01/06/2013
Serviço: 40001   DESMATAMENTO, DESTOCAMENTO E LIMPEZA - ÁRVORES COM DIÂMETROS MENORES DE 15 CM Unidade: m2
(A)Equipamento Código
auxiliarUt. Pr Ut. Impr Vl. Hr. Prod Vl. Hr. Imp Consumo Custo Horário
TRATOR ESTEIRAS COM LAMINA - CAT D-6
OU EQUIVALENTE30000 1,0000 0,0000 183,65 73,10 1,0000 183,65
(A)Total: 183,65
(B)Mão-de-Obra Código
auxiliarEq. Salarial Sal/Hora Encargos(%) Consumo Custo Horário
ENCARREGADO DE SERVIÇO 20002 3,30 19,51 91,86 0,4000 7,80
['ENCARREGADO', 'DE', 'SERVIÇO', '20002', '3,30', '19,51', '91,86', '0,4000', '7,80']
----> mdo: ['ENCARREGADO DE SERVIÇO', '20002', '3,30', '19,51', '91,86', '0,4000', '7,80']
AJUDANTE 20003 1,12 6,62 91,86 2,0000 13,24
['AJUDANTE', '20003', '1,12', '6,62', '91,86', '2,00

In [181]:
global_mao_de_obra

[['40001',
  'ENCARREGADO DE SERVIÇO',
  '20002',
  '3,30',
  '19,51',
  '91,86',
  '0,4000',
  '7,80'],
 ['40001', 'AJUDANTE', '20003', '1,12', '6,62', '91,86', '2,0000', '13,24'],
 ['40002',
  'ENCARREGADO DE SERVIÇO',
  '20002',
  '3,30',
  '19,51',
  '91,86',
  '0,4000',
  '7,80'],
 ['40002', 'AJUDANTE', '20003', '1,12', '6,62', '91,86', '2,0000', '13,24'],
 ['40005',
  'ENCARREGADO DE SERVIÇO',
  '20002',
  '3,30',
  '19,51',
  '91,86',
  '0,2000',
  '3,90'],
 ['40005', 'AJUDANTE', '20003', '1,12', '6,62', '91,86', '1,0000', '6,62'],
 ['40010',
  'ENCARREGADO DE SERVIÇO',
  '20002',
  '3,30',
  '19,51',
  '91,86',
  '0,3000',
  '5,85'],
 ['40010', 'AJUDANTE', '20003', '1,12', '6,62', '91,86', '1,0000', '6,62'],
 ['40011',
  'ENCARREGADO DE SERVIÇO',
  '20002',
  '3,30',
  '19,51',
  '91,86',
  '0,3000',
  '5,85'],
 ['40011', 'AJUDANTE', '20003', '1,12', '6,62', '91,86', '1,0000', '6,62'],
 ['40012',
  'ENCARREGADO DE SERVIÇO',
  '20002',
  '3,30',
  '19,51',
  '91,86',
  '1,0000',

In [55]:
sheets_dict = pd.read_excel("source/edital_pav_rod_sem_mob.xlsx", sheet_name=None)

all_sheets = []
for name, sheet in sheets_dict.items():
    sheet["sheet"] = name
    sheet = sheet.rename(columns=lambda x: x.split("\n")[-1])
    all_sheets.append(sheet)

full_table = pd.concat(all_sheets)
full_table.reset_index(inplace=True, drop=True)

In [56]:
full_table.rename(
    columns={
        "PROJETO BÁSICO DE ENGENHARIA": "servico",
        "Unnamed: 1": "descricao",
        "Unnamed: 2": "unidade",
        "Unnamed: 3": "quantidade",
        "Unnamed: 4": "preco unitario",
        "Unnamed: 5": "preco total",
        "sheet": "projeto",
    },
    inplace=True,
)

In [57]:
full_table["servico"] = full_table["servico"].astype(str).str.strip()

In [58]:
df = full_table[
    (~full_table["servico"].str.contains("TOTAL"))
    & (full_table["servico"].str.len() == 5)
]

In [59]:
df

,servico,descricao,unidade,quantidade,preco unitario,preco total,projeto
4,40005,"Desmatamento, destocamento e limpeza (arv. até...",m²,350400,0.19,66576,NS01
5,40015,Carga de entulhos - material de limpeza,m³,91104,1.19,108413.76,NS01
6,40020,Transporte de entulhos - material de limpeza (...,m³.km,455520,1.01,460075.2,NS01
7,40025,Escavação e carga de material de 1ª categoria,m³,557909.7758,4.68,2611017.75,NS01
8,40050,Escavação e carga de solo mole,m³,9647.2,8.11,78238.79,NS01
...,...,...,...,...,...,...,...
1053,42805,"Gabiões (0,50 m)",m³,81.04,416.7,33769.36,Vila Maria
1054,42800,"Gabiões (1,00 m)",m³,214.14,338.04,72387.88,Vila Maria
1055,42830,Concreto fck=11MPa,m³,7.86,290.49,2283.25,Vila Maria
1056,45550,Meio Fio sem sarjeta (GAP),m,14640,14.86,217550.4,Vila Maria


In [60]:
for source in composicoes_files:
    print(source)

source/composicoes.pdf
source/composicoes---t109.pdf


In [61]:
reader_composicoes = PdfReader("source/composicoes.pdf")
reader_composicoes_t109 = PdfReader("source/composicoes---t109.pdf")
projetos = {}
for projeto in df["projeto"].unique():
    writer = PdfWriter()
    servicos = df[(df["projeto"] == projeto)]["servico"].to_list()
    pages_pdf_composicoes = []
    pages_pdf_composicoes_t109 = []
    print(projeto, servicos)
    for servico in servicos:
        if servico in global_composicoes.get("source/composicoes.pdf").keys():
            pdf_page = global_composicoes.get("source/composicoes.pdf").get(
                servico, servico
            )
            pages_pdf_composicoes.append(pdf_page)
            writer.add_page(reader_composicoes.pages[pdf_page])
            print("composicoes:", pdf_page)

        elif servico in global_composicoes.get("source/composicoes---t109.pdf").keys():
            pdf_page = global_composicoes.get("source/composicoes---t109.pdf").get(
                servico, servico
            )
            pages_pdf_composicoes_t109.append(pdf_page)
            writer.add_page(reader_composicoes_t109.pages[pdf_page])
            print("composicoes---t109:", pdf_page)
        # print(global_composicoes.get("source/composicoes.pdf").get(servico, servico))
    print(pages_pdf_composicoes, pages_pdf_composicoes_t109)
    projetos.update(
        {
            projeto: {
                "composicoes": pages_pdf_composicoes,
                "composicoes---t109": pages_pdf_composicoes_t109,
            }
        }
    )
    writer.write(f"target/projetos/{projeto}.pdf")
print(projetos)

NS01 ['40005', '40015', '40020', '40025', '40050', '40055', '40065', '40075', '40080', '40085', '40095', '40300', '40305', '40310', '40335', '40316', '40320', '40350', '40316', '40320', '40380', '40385', '40405', '40435', '40450', '40460', '41290', '47023', '45430', '45435', '45445', '45450', '45455', '45460', '45465', '45490', '45500', '45510', '45520', '45495', '45505', '45515', '45525', '45530', '45535', '45585', '45580', '45590', '45605', '40055', '42810', '42805', '42800', '45585', '42830', '45555', '45550', '41255', '41200', '41210', '41290', '41270', '40805', '40885', '40310', '42820', '40425', '40015', '40020', '40100', '40810', '40820', '40830', '40845', '40855', '45020', '45025', '45030', '45052', '45140', '45145', '45155', '45160', '45205', '45210', '45240', '45245', '45020', '45025', '45030', '45052', '45140', '45145', '45155', '45160', '45205', '45210', '45240', '45245', '45020', '45025', '45030', '45052', '45140', '45145', '45155', '45160', '45205', '45210', '45240', '452

In [62]:
sheets = df["projeto"].unique()

In [63]:
sheets

array(['NS01', 'NS03', 'NS05', 'Jardim Bonança', 'Parque Santa Cecília',
       'Vila Maria'], dtype=object)

In [64]:
df

,servico,descricao,unidade,quantidade,preco unitario,preco total,projeto
4,40005,"Desmatamento, destocamento e limpeza (arv. até...",m²,350400,0.19,66576,NS01
5,40015,Carga de entulhos - material de limpeza,m³,91104,1.19,108413.76,NS01
6,40020,Transporte de entulhos - material de limpeza (...,m³.km,455520,1.01,460075.2,NS01
7,40025,Escavação e carga de material de 1ª categoria,m³,557909.7758,4.68,2611017.75,NS01
8,40050,Escavação e carga de solo mole,m³,9647.2,8.11,78238.79,NS01
...,...,...,...,...,...,...,...
1053,42805,"Gabiões (0,50 m)",m³,81.04,416.7,33769.36,Vila Maria
1054,42800,"Gabiões (1,00 m)",m³,214.14,338.04,72387.88,Vila Maria
1055,42830,Concreto fck=11MPa,m³,7.86,290.49,2283.25,Vila Maria
1056,45550,Meio Fio sem sarjeta (GAP),m,14640,14.86,217550.4,Vila Maria


In [65]:
df_servico = pd.DataFrame(global_servico, columns=composicoes["servico_title"])

In [66]:
composicoes["equipamentos_title"]

['servico',
 'equipamento',
 'codigo',
 'ut. pr.',
 'ut. impr.',
 'vl. hr. prod.',
 'vl. hr. impr.',
 'consumo',
 'custo horario']

In [67]:
global_equipamentos

[['40001', '', 'TRATOR', 'ESTEIRAS', 'COM', 'LAMINA', '-', 'CAT', 'D-6'],
 ['40001',
  'OU',
  'EQUIVALENTE30000',
  '1,0000',
  '0,0000',
  '183,65',
  '73,10',
  '1,0000',
  '183,65'],
 ['40002', 'TRATOR', 'ESTEIRA', 'C/', 'LÂMINA', '-', 'CAT', 'D8', 'OU'],
 ['40002',
  '',
  'EQUIVALENTE30001',
  '1,0000',
  '0,0000',
  '327,93',
  '108,94',
  '1,0000',
  '327,93'],
 ['40005', 'CARREGADEIRA', 'DE', 'PNEUS', 'CAT', '-', '950', 'H', 'OU'],
 ['40005',
  '',
  'EQUIVALENTE30007',
  '1,0000',
  '0,0000',
  '185,85',
  '76,54',
  '1,0000',
  '185,85'],
 ['40006',
  'CAMINHÃO BASCULANTE 10 M3 - 15 T',
  '30037',
  '1,0000',
  '0,0000',
  '130,87',
  '42,43',
  '1,0000',
  '130,87'],
 ['40010', 'TRATOR', 'ESTEIRA', 'C/', 'LÂMINA', '-', 'CAT', 'D8', 'OU'],
 ['40010',
  '',
  'EQUIVALENTE30001',
  '1,0000',
  '0,0000',
  '327,93',
  '108,94',
  '1,0000',
  '327,93'],
 ['40011', 'TRATOR', 'ESTEIRA', 'C/', 'LÂMINA', '-', 'CAT', 'D8', 'OU'],
 ['40011',
  '',
  'EQUIVALENTE30001',
  '1,0000',
  '

In [68]:
# df_servico = pd.DataFrame(global_servico, columns=composicoes["servico_title"])
df_equipamentos = pd.DataFrame(
    global_equipamentos, columns=composicoes["equipamentos_title"]
)

In [69]:
df_mao_de_obra = pd.DataFrame(
    global_mao_de_obra, columns=composicoes["mao_de_obra_title"]
)
df_itens_de_incidencia = pd.DataFrame(
    global_itens_de_incidencia, columns=composicoes["itens_de_incidencia_title"]
)
df_materiais = pd.DataFrame(global_materiais, columns=composicoes["materiais_title"])
df_servicos = pd.DataFrame(global_servicos, columns=composicoes["servicos_title"])
df_itens_de_transporte = pd.DataFrame(
    global_itens_de_transporte, columns=composicoes["itens_de_transporte_title"]
)

In [70]:
# df_servico
# df_equipamentos
# df_mao_de_obra
# df_itens_de_incidencia
# df_materiais
# df_servicos
df_itens_de_transporte

,servico,itens de transporte,codigo,unidade,formula,x1,x2,x3,custo unitario,consumo,custo
0,40480,,Custo,Adotado,"0,00",None,None,None,None,None,None
1,40480,,Preço,Unitário,Total,"0,00",None,None,None,None,None
2,40480,SCO - Sistema de Custos e Orçamentos,SCO,-,Sistema,de,Custos,e,Orçamentos,AGETOP,None
3,40490,,Custo,Adotado,"0,00",None,None,None,None,None,None
4,40490,,Preço,Unitário,Total,"0,00",None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
209,45207,SCO - Sistema de Custos e Orçamentos,SCO,-,Sistema,de,Custos,e,Orçamentos,AGETOP,None
210,47030,,Custo,Adotado,"1,31DT",None,None,None,None,None,None
211,47030,,"BDI:25,00%","0,32DT",None,None,None,None,None,None,None
212,47030,,Preço,Unitário,Total,"1,63DT",None,None,None,None,None


In [71]:
# df_servico.convert_dtypes().astype({"producao": "float"}).dtypes

# df_servico["producao"].str.replace(",", "").replace(".", ",").astype("float").dtypes

composicoes_float = {
    "servico": [
        "producao",
        "custo unitario execucao",
        "custo direto",
        "bdi",
        "preco unitario total",
    ],
    "equipamentos": [
        "ut. pr.",
        "ut. impr.",
        "vl. hr. prod.",
        "vl. hr. impr.",
        "consumo",
        "custo horario",
    ],
    "mao_de_obra": [
        "eq. salarial",
        "sal. hora",
        "consumo",
        "custo horario",
    ],
    "itens_de_incidencia": ["%", "custo"],
    "materiais": [
        "custo unitario",
        "consumo",
        "custo",
    ],
    "servicos": [
        "custo unitario",
        "consumo",
        "custo",
    ],
    "itens_de_transporte": [
        # "unidade",
        "x1",
        "x2",
        # "x3",
        "custo unitario",
        "consumo",
        "custo",
    ],
}

for k, v in composicoes_float.items():
    print(k)
    # display(globals()[f"df_{k}"])
    for _v in v:
        print(_v)
        globals()[f"df_{k}"][f"{_v}"] = (
            globals()[f"df_{k}"][f"{_v}"]
            .str.replace(",", "")
            .replace(".", ",")
            .astype("float")
        )

# dfs = [df_servico]
# for _df in dfs:
#     display(_df)
#     for col in _df.iloc[:,-7:-1]:
#         display(_df[col])
# for col in _df:
#     print(col)

servico
producao
custo unitario execucao
custo direto
bdi


ValueError: could not convert string to float: '008DT'

In [ ]:
for s in sheets:
    with pd.ExcelWriter(f"target/Projeto/{s}.xlsx") as writer:
        _df = df[df["projeto"] == s]
        _df_servico = pd.merge(
            _df[
                [
                    "servico",
                    "descricao",
                    # "unidade",
                    "quantidade",
                    "preco unitario",
                    "preco total",
                ]
            ],
            df_servico[
                [
                    "servico",
                    "producao",
                    "unidade",
                    "custo unitario execucao",
                    "custo direto",
                    "bdi",
                    "preco unitario total",
                    "pagina",
                    "arquivo",
                ]
            ],
            # how="outer",
            on="servico",
            validate="m:m",
        )
        _df_servico.to_excel(writer, sheet_name="servico")
        for k, v in composicoes.items():
            if k.replace("_title", "") != "servico":
                tipo = k.replace("_title", "")
                # print(tipo)
                # display(globals()[f"df_{k.replace('_title', '')}"])
                # display(df[df["projeto"] == s])

                pd.merge(
                    _df_servico,
                    globals()[f"df_{tipo}"],
                    # how="outer",
                    on="servico",
                    validate="m:m",
                ).to_excel(writer, sheet_name=tipo)
    # globals()[f"global_{k.replace('_title', '')}"df[df["projeto"] == s])

In [ ]:
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook

wb = load_workbook(filename="target/Projetos.xlsx")
for s in sheets:
    _df = df[df["projeto"] == s]
    ws = wb[s]
    for k, v in composicoes.items():
        tipo = k.replace("_title", "")
        for r in dataframe_to_rows(
            pd.merge(
                _df,
                globals()[f"df_{tipo}"],
                # how="outer",
                on="servico",
                validate="m:m",
            ),
            index=False,
            header=True,
        ):  # No index and don't append the column headers
            ws.append(r)
        wb.save("target/Projetos.xlsx")